# Runtime -> Change Runtime -> GPU

In [ ]:
# check for the GPU provided in the runtime
!nvidia-smi
# using quiet method for controlling the log
# for suppressing the colored errors and warning in the terminal
!pip install --quiet transformers==4.1.1
# pytorch lightning for smoother model training and data loading
!pip install --quiet https://github.com/PyTorchLightning/pytorch-lightning/releases/download/1.6.1/pytorch-lightning-1.6.1.tar.gz 
# using HuggingFace tokenizers
!pip install --quiet tokenizers==0.9.4
# Google's sentencepiece
!pip install --quiet sentencepiece==0.1.94
# mostly pl is used while doing complex model training
import pytorch_lightning as pl
print(pl.__version__)
# argparse makes it easier to write user friendly command line interfaces
import argparse
# package for faster file name matching
import glob
# makiing directories for data 
import os
# reading json files as the data is present in json files
import json
# time module for calculating the model runtime
import time
# Allows writing status messages to a file
import logging
# generate random float numbers uniformly
import random
# regex module for text 
import re
# module provides various functions which work on 
# iterators too produce complex iterators
from itertools import chain
from string import punctuation
# pandas for data manipulation
import pandas as pd
# numpy for array operations
import numpy as np
# PyTorch
import torch
# provides various classes representing file system paths
# with appropriate semantics
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
# splitting the data 
from sklearn.model_selection import train_test_split
# ANSII color formatting for ouput in terminal
from termcolor import colored
# wrapping paragraphs into string
import textwrap
# model checkpoints in pretrained model
from pytorch_lightning.callbacks import ModelCheckpoint
'''
optimizer - AdamW
T5 Conditional Generator in which we'll give conditions
T5 tokenizer because it is fast
training the model without a learning rate
'''
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
# Seeds all the processes including numpy torch and other imported modules.
pl.seed_everything(0)
# check the version provided by Lightning
import pytorch_lightning as pl
print(pl.__version__)

Tue May  3 06:27:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Global seed set to 0


1.6.1


# 데이터셋

In [ ]:
import os
import json
import numpy as np
from tqdm.notebook import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/6조_파이널PJT/data/ko1.0_add_ko2.0.csv')

# 답변이 본문에 알맞은 위치인지 확인

In [ ]:
df.shape
len(df.question.unique())
len(df.context.unique())
sample_question = df.iloc[0]
sample_question

def color_answer(question):
  answer_start, answer_end = question["answer_start"],question["answer_end"]
  context = question['context']
  return  colored(context[:answer_start], "white") + \
    colored(context[answer_start:answer_end], "green") + \
    colored(context[answer_end+1:], "white")
print(sample_question['question'])
print()
print("Answer: ")
for wrap in textwrap.wrap(color_answer(sample_question), width = 100):
  print(wrap)

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?

Answer: 
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡 쓰려는 뜻을 갖는다. 이 시기
바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로
파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은
의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼
수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의
연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고
말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도
있다.


# 토큰화

In [ ]:
MODEL_NAME ='KETI-AIR/ke-t5-base' 
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
sample_encoding = tokenizer('is the glass half empty or half full?', 'It depends on the initial state of the glass. If the glass starts out empty and liquid is added until it is half full, it is half full. If the glass starts out full and liquid is removed until it is half empty, it is half empty.')
sample_encoding.keys()
print(sample_encoding["input_ids"])
print(sample_encoding["attention_mask"])
print(len(sample_encoding['input_ids']), len(sample_encoding['attention_mask']))

# Checking the decoding of the input ids
preds = [
        tokenizer.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for input_id in sample_encoding['input_ids']
]
preds= " ".join(preds)
for wrap in textwrap.wrap(preds, width = 80):
  print(wrap)
encoding = tokenizer(
    sample_question['question'],
    sample_question['context'],
    max_length=396,
    padding='max_length',
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
encoding.keys()
tokenizer.special_tokens_map
tokenizer.eos_token, tokenizer.eos_token_id
tokenizer.decode(encoding['input_ids'].squeeze()) 

Downloading:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

[43, 5, 12857, 1648, 19876, 117, 1648, 1479, 85, 1, 223, 26000, 46, 5, 9113, 604, 14, 5, 12857, 3, 799, 5, 12857, 9162, 176, 19876, 13, 24483, 43, 1586, 1429, 60, 43, 1648, 1479, 4, 60, 43, 1648, 1479, 3, 799, 5, 12857, 9162, 176, 1479, 13, 24483, 43, 11514, 1429, 60, 43, 1648, 19876, 4, 60, 43, 1648, 19876, 3, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
63 63
is the glass half empty or half full ? </s> It depends on the initial state of
the glass . If the glass starts out empty and liquid is added until it is half
full , it is half full . If the glass starts out full and liquid is removed
until it is half empty , it is half empty . </s>


'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?</s> 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 <unk> 은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

# 답을 위한 라벨링 확인

In [ ]:
answer_encoding = tokenizer(
    sample_question['answer_text'],
    max_length=32,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
tokenizer.decode(answer_encoding['input_ids'].squeeze())
labels = answer_encoding["input_ids"]
labels
labels[labels == 0] = -100
labels 

tensor([[58337,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100]])

# 모델에 인풋에 맞게 데이터셋 만들기
길이, 토큰화

In [ ]:
class KorQuADataset(Dataset):
  def __init__(
      self,
      data:pd.DataFrame,
      tokenizer:T5Tokenizer,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,
      ):
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len
  def __len__(self):
    return len(self.data)
  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]
    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_len,
      padding='max_length',
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length=self.target_max_token_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    labels = target_encoding['input_ids']
    labels[labels==0] = -100
    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )
sample_dataset = KorQuADataset(df, tokenizer)
for data in sample_dataset:
  print("Question: ", data['question'])
  print("Answer text: ", data['answer_text'])
  print("Input_ids: ", data['input_ids'][:10])
  print("Labels: ", data['labels'][:10])
  break 

Question:  바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
Answer text:  교향곡
Input_ids:  tensor([  396,   775, 21778, 63122,     6,  1649,  8643, 15171, 12022,  6937])
Labels:  tensor([58337,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100])


# Train과 Test set 만들기

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.05)
train_df.shape,  val_df.shape 

((96849, 5), (5098, 5))

# 파이토치 라이트닝 모듈 만들기

In [ ]:
 class KorDataModule(pl.LightningDataModule):
   def __init__(
       self,
       train_df: pd.DataFrame,
       test_df: pd.DataFrame,
       tokenizer:T5Tokenizer,
       batch_size: int = 8,
       source_max_token_len: int = 396,
       target_max_token_len: int = 32,
       ):
     super().__init__()
     self.train_df = train_df
     self.test_df = test_df
     self.tokenizer = tokenizer
     self.batch_size = batch_size
     self.source_max_token_len = source_max_token_len
     self.target_max_token_len = target_max_token_len
   def setup(self):
     self.train_dataset = KorQuADataset(
         self.train_df,
         self.tokenizer,
         self.source_max_token_len,
         self.target_max_token_len
         )
     self.test_dataset = KorQuADataset(
     self.test_df,
     self.tokenizer,
     self.source_max_token_len,
     self.target_max_token_len
     )
   def train_dataloader(self):
     return DataLoader(
         self.train_dataset,
         batch_size=self.batch_size,
         shuffle=True,
         num_workers=4
         )
   def val_dataloader(self):
     return DataLoader(
         self.test_dataset,
         batch_size=self.batch_size,
         num_workers=4
         )
   def test_dataloader(self):
     return DataLoader(
         self.test_dataset,
         batch_size=1,
         num_workers=4
         )
 BATCH_SIZE = 4
 N_EPOCHS = 6
 data_module = KorDataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
 data_module.setup() 

# T5ForConditionalGeneration 모델을 이용해 파이토치 라이트닝 빌드

In [ ]:
class KorQAModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)
    return output.loss, output.logits
  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions":outputs, "labels": labels}
  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss
  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=0.0001)
    return optimizer
model = KorQAModel()

Downloading:   0%|          | 0.00/599 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

# 우리의 데이터 셋으로 파인튜닝된 파이토치 라이트닝으로 학습

In [ ]:
model = KorQAModel()
# To record the best performing model using checkpoint
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/MyDrive/final_pjt/ET5/model",
    filename="best-checkpoint-test",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

trainer = pl.Trainer(
    checkpoint_callback=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate = 30
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f4a6b9daa90>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f4a6b9daa90>)`.
  # remove all callbacks with a type that occurs in model callbacks
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  refresh_rate = 1 if r

# 텐서보드로 확인하며 학습

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

trainer.fit(model, data_module)
trainer.test()  # evaluate the model according to the last checkpoint

In [ ]:
trained_model = KorQAModel.load_from_checkpoint('/content/drive/MyDrive/final_pjt/ET5/model/best-checkpoint.ckpt')
trained_model.freeze() # 

# 질문에 적절한 내용 찾기 함수


In [ ]:
dat = pd.read_csv('/content/drive/MyDrive/6조_파이널PJT/data/data_add_title.csv')
dat.head()

,question,context,answer_text,answer_start,answer_end,title
0,드라마 예고범의 감독은 누구일까?,예고범 일본어 予告犯 은 츠츠이 테츠야의 만화 작품이다 점프 카이 슈에이샤 에서 2...,나카무라 요시히로 히라바야시 카츠토시 사와다 메구미,6302,6332,예고범
1,심규언은 17대 지방 선거에서 몇 표를 득표하였는가?,심규언 沈圭彦 1955년 10월 1일 은 대한민국의 제17·18대 강원도 동해시장이...,20890 표,3873,3881,심규언
2,장기이식으로 인해 얻을 수 있는 효과는?,장기 이식 臟器移植 이란 어떤 조직 또는 장기의 파손된 기능을 대체할 목적으로 원래...,자유 시장이 장기 이식에서의 희소성 문제를 해결할 수 있다고 말했다,6469,6507,장기_이식
3,'망언' 논란을 야기한 천안함 사태에 대한 이윤성의 발언은 무엇이며 어떤 비판을 받...,이윤성 李允盛 1944년 10월 2일 은 前 kbs 앵커이자 前 무소속 국회의원이다...,전국 동시 지방 선거를 앞두고 있던 2010년 5월 31일 이윤성 의원은 여의도 당...,5818,6699,이윤성_(1944년)
4,2005년 행정부의 6급이하 공무원은 전체에 몇퍼정도를 차지하나요?,이윤성 李允盛 1944년 10월 2일 은 前 kbs 앵커이자 前 무소속 국회의원이다...,전체의 95정도,7770,7779,이윤성_(1944년)


In [ ]:
# https://github.com/monologg/KoBERT-Transformers/blob/master/kobert_transformers/tokenization_kobert.py 에서 제공하는
# tokenization_kobert.py 를 현재 코드 위치에 해두기
import tokenization_kobert

In [ ]:
from tokenization_kobert import KoBertTokenizer

In [ ]:
tokenizer_t = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
!pip install tensorflow #텐서플로우 설치
!pip install sklearn #사이킷런 설치
!pip install konlpy

     |████████████████████████████████| 462 kB 7.9 MB/s 
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 71.1 MB/s 


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
okt = Okt()
import numpy as np
import pandas as pd
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

In [ ]:
def in_title(question, dat):
  dat = dat.drop_duplicates(['context'], ignore_index = True)
  t_list = []
  con_list = []
  for i in range(len(dat)):
    if dat['title'][i].split('(')[0] in question.replace(' ', ''):  
      contet = dat.loc[i,:]
      t_list.append(contet)

    else:
      pass
  t_list = pd.DataFrame(t_list, columns = dat.columns)
  con_list = dat[dat['context'].str.contains(okt.nouns(question)[0])]
  use = pd.concat([t_list, con_list], ignore_index=True)
  if len(use) >0:
    return use
  else :
    return print('데이터 없음')


In [ ]:
def title_re(t_list, question):     

  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['question'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      tfidf_normalized = tfidf_matrix/np.sum(tfidf_matrix)
      manhattan_d = manhattan_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])
      t_list['score'][i] = manhattan_d[0][0]
  t_list = t_list[t_list['score'] == min(t_list['score'])]
  return t_list


In [ ]:
def cosin_sim(t_list, question):
  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['context'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
      t_list['score'][i] = cos_similar[0][0]
  # return cos_similar[0][0], type(cos_similar[0][0])
  return t_list.loc[t_list['score'].idxmax()]['context']

In [ ]:
def anw_test(context, question):

  source_encoding=tokenizer(
    question,
    context,
    max_length = 396,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
  generated_ids = trained_model.model.generate(
    input_ids=source_encoding["input_ids"],
    attention_mask=source_encoding["attention_mask"],
    num_beams=1,  # greedy search
    max_length=80,
    repetition_penalty=2.5,
    early_stopping=True,
    use_cache=True)
  preds = [
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids]
  return "".join(preds), context 


In [ ]:
def last(question, dat):
  # dat = dat.drop_duplicates(['context'], ignore_index = True)
  a = in_title(question,dat)   ## 타이틀, 컨텍스트 
  b= title_re(a, question)    ## 질문,타이틀 유사도 
  context =cosin_sim(b, question)    
  answer = anw_test(context, question)   ## 정답 예측, 컨텍스트 
  return answer

In [ ]:
last('인공심장으로 297일 동안 살아있었던 동물은 무엇인가?', dat)

### 예측 정답과 실제 정답 비교용 데이터프레임 생성 


In [ ]:
sample_df = dat.sample(n = 100 , random_state=15)
re_sample = sample_df.reset_index()
ans_list = []
context_list = []

for i in range(100): 
  try:
    answer = last(re_sample.loc[i,'question'], re_sample)
    ans_list.append(answer)
    # for i in answer: 
    #   ans_list.append(a)
    #   i+=1
    #   context_list.append(a)
  except:
    ans_list.append('NaN')
    context_list.append('NaN')

In [ ]:
new_df = re_sample.copy()
for i in ans_list: 
  new_df['pred_ans'] = ans_list

new_df.drop(['answer_start','answer_end'], axis=1)

In [ ]:
@misc{ke_t5,
    author       = {KETI AIRC},
    title        = {KE-T5: Korean English T5},
    month        = may,
    year         = 2022,
    url          = {https://github.com/AIRC-KETI/ke-t5}
}